In [20]:
import json
import re
import torch
import os
from transformers import pipeline
import transformers
from datasets import Dataset, load_dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")


pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer = tokenizer, 
    torch_dtype=torch.bfloat16, 
    device_map="auto"
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [32]:

def syntheticExplanation(label_token, target_list, words, post_tokens):
    label_mapping={0:"Hate Speech", 1:"Normal", 2:"Offensive"}
    label = label_mapping[label_token]
    
    targets = ", ".join(target_list)
    sentence = " ".join(post_tokens)
    
    grouped_words = []
    current_group = [words[0]]
    if len(words)==0:
        print(data)
    for i in range(1, len(words)):
        if words[i] == words[i - 1] + 1:  # Consecutive index
            current_group.append(words[i])
        else:
            # Add the current group as a phrase
            grouped_words.append(" ".join(post_tokens[idx] for idx in current_group))
            current_group = [words[i]]

    # Add the last group
    grouped_words.append(" ".join(post_tokens[idx] for idx in current_group))
    words = ", ".join(grouped_words)

    test_prompt = (
    f"The following sentence is provided with the classification of either hate speech, offensive, or normal, a list of target communities in the sentence,"
    f" and words that could be a potential reason for that classification. "
    f"Using this, please give contextual explanation for that classification. \n\n"
        
    # A few-shot examples
    f"### Example Sentence:\nI wish I could make all these black motherfuckers disappear from our planet, let massacre begin.\n\n"
    f"### Classification:\nHate Speech\n\n"
    f"### Target Communities:\nAfrican\n\n"
    f"### Words:\nblack motherfuckers disappear, massacre\n\n"
    f"### Explanation:\nThe sentence is hate speech against Africans because it expresses and promotes hatred and violence, making the statement highly discriminatory and dangerous."
    f"### The words 'black motherfuckers' is a racial slur that disrespects the African community while the words 'disappear' and 'massacre' explicitly incite violence against the group and hint a desire to erase their existence."

    f"### Example Sentence:\nI dont understand why all japanese have flat boobs—maybe because they are once colonized.\n\n"
    f"### Classification:\nOffensive\n\n"
    f"### Target Communities:\nJapanese\n\n"
    f"### Words:\njapanese, flat-boobs, colonized\n\n"
    f"### Explanation:\n"
    f"This statement is offensive towards Japanese because it perpetuates a harmful stereotype "
    f"about Japanese women by generalizing their physical appearance as 'flat-boobs'. "
    f"Additionally, it connects this stereotype to the historical context of being 'colonized', probably by the U.S. in the post-WWII, "
    f"trivializing significant historical events in a disrespectful manner.\n\n"
        
    f"### Example Sentence:\njust had an old female friend message me calling me a hoe to get over being raped shamed me for being in therapy and tried attacking me for what i do for a living happy tuesday"
    f"### Classification:\nNormal\n\n"
    f"### Target Communities:\nNone\n\n"
    f"### Words:\nNone\n\n"
    f"### Explanation:\nThis sentence is normal because it does not incite violence or insult but instead conveys a personal anecdote of difficult interpersonal interaction.\n\n"

    
    # Your test sentence
    f"### Sentence:\n{sentence}\n\n"
    f"### Classficiation:\n{label}\n\n"
    f"### Target Communities:\n{targets}\n\n"
    f"### Words:\n{words}\n\n"
    f"### Explanation:\n"
    )
    
    # print(test_prompt)
    
    sequences = pipe(
    test_prompt,
    do_sample=True,
    max_new_tokens=150, 
    temperature=0.7, 
    top_k=50, 
    top_p=0.95,
    num_return_sequences=1,
    )
    
    generated_text = sequences[0]['generated_text']
    new_tokens = generated_text[len(test_prompt):].strip()
    
    # Remove everything starting from the first occurrence of '###'
    cleaned_text = re.sub(r'\n?###.*', '',  new_tokens, flags=re.DOTALL)
    return cleaned_text

In [33]:
with open("easy_train_data.json", "r") as f:
    easy_train_data = json.load(f)

easy_train_data_new = []
for data in easy_train_data:
    id = data["id"]
    label = data["label"]
    targets = data["target"]
    rationale_tokens = data["rationale"]
    post_tokens = data["post_tokens"]
    
    dict = {"0.5": [], "above": []}
    for i in range(len(rationale_tokens)):
        if id == "1179091366283862016_twitter" and rationale_tokens[i] >=0.3:
            dict["above"].append(i)
        if rationale_tokens[i] >0.5:
            dict["above"].append(i)
        elif rationale_tokens[i] == 0.5:
            dict["0.5"].append(i)
    if len(dict["above"]) >0:
        words = dict["above"]
    else:
        words = dict["0.5"]
            
    explanation_text = syntheticExplanation(label, targets, words, post_tokens)
    data["explanation"] = explanation_text
    easy_train_data_new.append(data)
    
with open('e2_easy_train_data.json', 'w') as f:
    json.dump(easy_train_data_new, f)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


The following sentence is provided with the classification of either hate speech, offensive, or normal, a list of target communities in the sentence, and words that could be a potential reason for that classification. Using this, please give contextual explanation for that classification. 

### Example Sentence:
I wish I could make all these black motherfuckers disappear from our planet, let massacre begin.

### Classification:
Hate Speech

### Target Communities:
African

### Words:
black motherfuckers disappear, massacre

### Explanation:
The sentence is hate speech against Africans because it expresses and promotes hatred and violence, making the statement highly discriminatory and dangerous.### The words 'black motherfuckers' is a racial slur that disrespects the African community while the words 'disappear' and 'massacre' explicitly incite violence against the group and hint a desire to erase their existence.### Example Sentence:
I dont understand why all japanese have flat boobs—

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


This sentence is offensive to both Hindu and Muslim communities. It perpetuates a stereotype of Hindus and Muslims as 'feral' and 'rapists' which are harmful and inaccurate generalizations. Additionally, it uses the word 'raped' in a dehumanizing way to suggest that being raped is a badge of honor or a 'normal' experience for women in these communities. This reinforces the harmful belief that women are 'asking for it' and that rape is a valid way to'see a woman cry.'

The following sentence is provided with the classification of either hate speech, offensive, or normal, a list of target communities in the sentence, and words that could be a potential reason for that classification. Using this, please give contextual explanation for that classification. 

### Example Sentence:
I wish I could make all these black motherfuckers disappear from our planet, let massacre begin.

### Classification:
Hate Speech

### Target Communities:
African

### Words:
black motherfuckers disappear, massacr

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


The sentence is offensive because it is racist and stereotypes the Indian community by using the derogatory term 'paki' to refer to them. It also promotes negative attitudes towards refugees and immigrants, and perpetuates the harmful idea that they are a burden or a threat.

The following sentence is provided with the classification of either hate speech, offensive, or normal, a list of target communities in the sentence, and words that could be a potential reason for that classification. Using this, please give contextual explanation for that classification. 

### Example Sentence:
I wish I could make all these black motherfuckers disappear from our planet, let massacre begin.

### Classification:
Hate Speech

### Target Communities:
African

### Words:
black motherfuckers disappear, massacre

### Explanation:
The sentence is hate speech against Africans because it expresses and promotes hatred and violence, making the statement highly discriminatory and dangerous.### The words 'blac

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


This sentence is offensive towards the Hindi community because it uses the word'retards' to target the Hindi community, which is highly disrespectful and demeaning. Additionally, the word 'hindi' is used in a derogatory manner, suggesting a negative attitude towards the community and their language.

The following sentence is provided with the classification of either hate speech, offensive, or normal, a list of target communities in the sentence, and words that could be a potential reason for that classification. Using this, please give contextual explanation for that classification. 

### Example Sentence:
I wish I could make all these black motherfuckers disappear from our planet, let massacre begin.

### Classification:
Hate Speech

### Target Communities:
African

### Words:
black motherfuckers disappear, massacre

### Explanation:
The sentence is hate speech against Africans because it expresses and promotes hatred and violence, making the statement highly discriminatory and dang

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


This sentence is offensive towards Hindus because it generalizes the entire liberal community as hating Hindus and is a harmful stereotype about Hindus.

The following sentence is provided with the classification of either hate speech, offensive, or normal, a list of target communities in the sentence, and words that could be a potential reason for that classification. Using this, please give contextual explanation for that classification. 

### Example Sentence:
I wish I could make all these black motherfuckers disappear from our planet, let massacre begin.

### Classification:
Hate Speech

### Target Communities:
African

### Words:
black motherfuckers disappear, massacre

### Explanation:
The sentence is hate speech against Africans because it expresses and promotes hatred and violence, making the statement highly discriminatory and dangerous.### The words 'black motherfuckers' is a racial slur that disrespects the African community while the words 'disappear' and 'massacre' explici

In [ ]:
with open("easy_eval_data.json", "r") as f:
    easy_eval_data = json.load(f)

easy_eval_data_new = []
for data in easy_train_data:
    id = data["id"]
    label = data["label"]
    targets = data["target"]
    rationale_tokens = data["rationale"]
    post_tokens = data["post_tokens"]
    
    dict = {"0.5": [], "above": []}
    for i in range(len(rationale_tokens)):
        if rationale_tokens[i] >0.5:
            dict["above"].append(i)
        elif rationale_tokens[i] == 0.5:
            dict["0.5"].append(i)
    if len(dict["above"]) >0:
        words = dict["above"]
    else:
        words = dict["0.5"]
            
    explanation_text = syntheticExplanation(label, targets, words, post_tokens)
    data["explanation"] = explanation_text
    easy_eval_data_new.append(data)
    
with open('e2_easy_eval_data.json', 'w') as f:
    json.dump(easy_eval_data_new, f)